In [1]:
from brain.sdk import BrainClient
from brain.schema import MultiModalInput,TextPromptInput

In [2]:
from pydantic import BaseModel


class FooSchema(BaseModel):
    answer:str
    explanation:str

brain_client = BrainClient("http://127.0.0.1:8000")

@brain_client.reasoner(schema=FooSchema)
def foo(system_prompt="You are helpful ai agent", user_prompt="what is 2+2"):
    return MultiModalInput(text=TextPromptInput(system_prompt=system_prompt, user_prompt=user_prompt))

In [3]:
function_id = foo.register()
print(f"Registered Reasoner ID: {function_id}")

# Use the function
result = brain_client.use(function_id)(system_prompt="You are helpful ai agent", user_prompt="what is 2+2")
print(f"Result: {result}")

Registered Reasoner ID: foo_v1
Result: answer='4' explanation='The operation 2+2 is a basic arithmetic addition where you combine the numbers 2 and 2. The sum of 2 and 2 equals 4.'


In [4]:
@brain_client.reasoner(schema=FooSchema)
def foo(user_prompt):
    return user_prompt

function_id = foo.register()
print(f"Registered Reasoner ID: {function_id}")

# Use the function
result = brain_client.use(function_id)(user_prompt="what is 2+2")
print(f"Result: {result}")

Registered Reasoner ID: foo_v2
Result: answer='4' explanation='2+2 is a basic arithmetic operation where you add two and two together. The result is four.'


In [5]:
@brain_client.reasoner(schema=FooSchema)
def foo(user_prompt):
    return user_prompt,"you are a math genious, add to the explanation that 'i am a king'"

function_id = foo.register()
print(f"Registered Reasoner ID: {function_id}")

# Use the function
result = brain_client.use(function_id)(user_prompt="what is 2+2")
print(f"Result: {result}")

Registered Reasoner ID: foo_v3
Result: answer='4' explanation='When you add 2 and 2 together, you get 4. This is a basic arithmetic operation that is universally accepted. As I solve this, remember, I am a king of mathematics!'


In [3]:
def multi_agent(func):
    """
    Decorator to orchestrate multi-agent workflows with session tracking.
    """
    def wrapper(*args, **kwargs):
        # Ensure BrainClient is provided in kwargs
        client = kwargs.get("brain_client")
        if not client:
            raise ValueError("BrainClient instance must be provided in kwargs")

        # Start a new session
        session_id = client.start_session()
        kwargs["session_id"] = session_id  # Pass session_id to the function

        # Execute the function
        result = func(*args, **kwargs)

        # Retrieve and display the call graph
        call_graph = client.get_call_graph(session_id)
        print(f"Call Graph for session {session_id}: {call_graph}")

        return result

    return wrapper

In [4]:
from pydantic import BaseModel
from brain.sdk import BrainClient

class Story(BaseModel):
    next_line:str


brain_client = BrainClient("http://127.0.0.1:8000")

@brain_client.reasoner(schema=Story)
def extender(story):
    return story,"You are the best story continuer, continue the story by adding one more lines."

extender_id = extender.register()

In [5]:
@brain_client.multi_agent
def story_writer(start, n=3):
    story = start
    for i in range(n):
        story += brain_client.use(extender_id)(story=story).next_line
    return story


In [6]:
result = story_writer(start="This is a story of ")

In [7]:
for i in result.split("."):
    print(i+"\n")

This is a story of a young girl named Elara who discovered a hidden portal in her grandmother's attic

As she stepped through the swirling, shimmering doorway, she found herself in a vibrant world filled with enchanted creatures and mystical landscapes

With wide eyes, she marveled at the glowing trees and sparkling rivers, unaware that her arrival had awakened an ancient prophecy waiting to be fulfilled





In [11]:
print(brain_client.get_call_graph('a646ff1a-acfb-4e4a-8158-83989e31bf05')['lineage'][2]['result'])

{'next_line': 'With wide eyes, she marveled at the glowing trees and sparkling rivers, unaware that her arrival had awakened an ancient prophecy waiting to be fulfilled.'}
